In [11]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy.stats import linregress

In [21]:
PEARSONBOUND = 0.5

In [13]:
basicFileName="basic_data.csv"
basicDF=pd.read_csv('../data/'+basicFileName)
basicDF.drop_duplicates(inplace=True)
basicDF['Symbol'].to_csv('../data/'+'ETFs.csv',index=False)


In [14]:
def firstSelect(df):
    
    #Inception记录股票上市时间
    firstDF=df[['Symbol','Fund Name','Assets','Category','Index','Inception','Volume']]
    print(len(firstDF))
    firstDF.dropna(inplace=True,subset=['Category'])
    firstDF.dropna(inplace=True,subset=['Index'])
    firstDF.dropna(inplace=True,subset=['Assets'])
    print(len(firstDF))

    category_counts=firstDF['Category'].value_counts()
    firstDF=firstDF[firstDF['Category'].isin(category_counts[category_counts>5].index)]
    firstDF = firstDF.loc[firstDF.groupby('Index')['Assets'].idxmax()]

    # 重置索引（可选）
    firstDF = firstDF.reset_index(drop=True)

    print(len(firstDF))

    firstDF[['Index','Symbol']].to_csv("./111.csv")
    return firstDF
    

firstDF=firstSelect(basicDF)

3963
2136
1763


/tmp/ipykernel_3158792/2128863428.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firstDF.dropna(inplace=True,subset=['Category'])
/tmp/ipykernel_3158792/2128863428.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firstDF.dropna(inplace=True,subset=['Index'])
/tmp/ipykernel_3158792/2128863428.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firstDF.dropna(inplace=True,subset=['Assets'])


In [15]:
#计算每类股票的资产总和
def category_assets(df):
    require_colums= ["Category","Assets"]
    for col in require_colums:
        if col not in firstDF.columns:
            print("Error: Missing column "+col)
            return

    df["Assets"]=pd.to_numeric(df["Assets"],errors='coerce')
    categorys=df.groupby("Category")["Assets"].sum().reset_index()
    categorys=categorys.sort_values("Assets",ascending=False)
    
    print("\nNet Assets by Category:\n")
    
    
    categorys.to_csv("../data/category_assets.csv")
    return categorys

categorys=category_assets(firstDF)
categorys.to_csv("../data/categorys.csv")
print(categorys["Category"])
# print(categorys[categorys["Category"]=="Large Blend"])


Net Assets by Category:

38                Large Blend
39               Large Growth
40                Large Value
18        Foreign Large Blend
66                 Technology
               ...           
64        Tactical Allocation
53        Nontraditional Bond
60            Single Currency
67    Trading--Inverse Equity
14              Equity Hedged
Name: Category, Length: 73, dtype: object


In [19]:
#Setp3:删除流动性和资产小于 val 的基金



def filter_fund(df,categorys):
    #决定使用哪种方式保留ETF
    #0表示绝对数额，1表示百分比
    method=1
    
    
    
    filtered_df = df.copy()
    require_colums= ["Category","Assets","Volume"]
    for col in require_colums:
        if col not in firstDF.columns:
            print("Error: Missing column "+col)
            return

    totAssets=categorys["Assets"].sum()
    print("Total Assets: ",totAssets)
        
    cateVal={"-1":-1}
    for etf in df["Category"]:
        cateVal[etf]=categorys[categorys["Category"]==etf]["Assets"].values[0]
    
    df["calval"]=df["Category"].map(cateVal)
    
    # 保留Assets大于0.1%总资产的基金和大于5%该类资产的基金
    if(method==0):
        
        filtered_df = df[
            (df['Assets'] > 0.001 * totAssets) |
            (df['Assets'] > df['Category'].map(lambda ca: 0.05 * cateVal.get(ca, 0)))
        ]
    
    if(method==1):
        # 第一步：计算全局的 Assets 后 10% 阈值
        global_threshold = filtered_df['Assets'].quantile(0.10)

        # 第二步：对每个 Category 计算 Assets 后 25% 的阈值
        category_thresholds = filtered_df.groupby('Category')['Assets'].transform(lambda x: x.quantile(0.25))

        # 第三步：过滤掉满足任一条件的股票
        filtered_df = filtered_df[~((filtered_df['Assets'] <= category_thresholds) | (df['Assets'] <= global_threshold))]

    
    filtered_df = filtered_df[~filtered_df['Category'].str.contains('Leverage', na=False)]
    filtered_df.to_csv("../data/filter_fund.csv")
    
    filtered_df["Symbol"].to_csv("../data/ETF1s.csv",index=False)
    return filtered_df

secondDF=filter_fund(firstDF,categorys)
categorys=category_assets(firstDF)
print(len(secondDF))
secondDF["Symbol"].to_csv("./ETF2.csv",index=False)

Total Assets:  7760177616282.0

Net Assets by Category:

1258


In [38]:
# linear_regression(SPY,ETF)
# find alpha and beta and 皮尔逊相关系数

def linear_regression_time(basicDf1,basicDf2,st,et):
    st=pd.to_datetime(st).date()
    et=pd.to_datetime(et).date()
    
    # basicDf1["Date"]=pd.to_datetime(basicDf1["date"]).dt.date
    # basicDf2["Date"]=pd.to_datetime(basicDf2["date"]).dt.date
    basicDf1=basicDf1[(basicDf1["Date"]>=st) & (basicDf1["Date"]<=et)]
    basicDf2=basicDf2[(basicDf2["Date"]>=st) & (basicDf2["Date"]<=et)]
    # common_dates=set(basicDf1['Date']).intersection(set(basicDf2['Date']))
    # common_dates=sorted(list(common_dates))
    
    # df1=basicDf1[basicDf1['Date'].isin(common_dates)]
    # df2=basicDf2[basicDf2['Date'].isin(common_dates)]
    if(len(basicDf1)<=5 or len(basicDf2)<=5):
        return np.nan,np.nan,False
    
    basicDf1.reset_index(drop=True,inplace=True)
    basicDf2.reset_index(drop=True,inplace=True)
    df1=basicDf1[['Date','close']]
    df2=basicDf2[['Date','close']]
    
    df=pd.merge(df1[['Date','close']],df2[['Date','close']],on='Date',suffixes=('_1','_2'))
    
    
    df['return1']=df1['close'].pct_change()
    df['return2']=df2['close'].pct_change()
    
    
    df=df.dropna()
    x = sm.add_constant(df['return1'])
    y = df['return2']

    model = sm.OLS(y, x).fit()
    
    alpha = model.params['const']
    beta = model.params['return1']
    
    correlation=np.corrcoef(df['return1'],df['return2'])[0,1]
    
    
    # print("Alpha:", alpha)
    # print("Beta:", beta)
    # # 绘制 SPY 和 ETF 收益率的散点图和回归线
    # plt.figure(figsize=(10, 6))
    # plt.scatter(df['return1'], df['return2'], label='Data Points', alpha=0.6)
    # plt.plot(df['return1'], alpha + beta * df['return1'], color='red', label='Regression Line')

    # # 添加图表标题和标签
    # plt.title('ETF Returns vs SPY Returns (Alpha and Beta)', fontsize=16)
    # plt.xlabel('SPY Returns', fontsize=12)
    # plt.ylabel('ETF Returns', fontsize=12)
    # plt.legend()
    # plt.grid(True)
    # plt.show()
    return alpha,beta,(abs(correlation)>PEARSONBOUND)



def linear_regression(basicDf1,basicDf2):
    basicDf1["Date"]=pd.to_datetime(basicDf1["date"]).dt.date
    basicDf2["Date"]=pd.to_datetime(basicDf2["date"]).dt.date
    common_dates=set(basicDf1['Date']).intersection(set(basicDf2['Date']))
    common_dates=sorted(list(common_dates))
    
    df1=basicDf1[basicDf1['Date'].isin(common_dates)]
    df2=basicDf2[basicDf2['Date'].isin(common_dates)]
    
    
    df1.reset_index(drop=True,inplace=True)
    df2.reset_index(drop=True,inplace=True)
    
    
    alphas=[]
    betas=[]
    correlations=[]
    for year in range(2010,2026):
        st=str(year)+'-01-01'
        et=str(year)+'-12-31'
        a,b,c=linear_regression_time(df1,df2,st,et)
        alphas.append(a)
        betas.append(b)
        correlations.append(c)
    
    alphaBeta=pd.DataFrame({'Year':range(2010,2026),'Alpha':alphas,'Beta':betas,'Correlation':correlations})
    
    # print("Alpha:", alpha)
    # print("Beta:", beta)
    # # 绘制 SPY 和 ETF 收益率的散点图和回归线
    # plt.figure(figsize=(10, 6))
    # plt.scatter(df['return1'], df['return2'], label='Data Points', alpha=0.6)
    # plt.plot(df['return1'], alpha + beta * df['return1'], color='red', label='Regression Line')

    # # 添加图表标题和标签
    # plt.title('ETF Returns vs SPY Returns (Alpha and Beta)', fontsize=16)
    # plt.xlabel('SPY Returns', fontsize=12)
    # plt.ylabel('ETF Returns', fontsize=12)
    # plt.legend()
    # plt.grid(True)
    # plt.show()
    return alphaBeta



dfSPY=pd.read_csv("../data/daily_data/SPY_daily.csv")
dfQQQ=pd.read_csv("../data/daily_data/QQQ_daily.csv")
# a,b,c=linear_regression(dfSPY,dfVOO)
alphaBeta=linear_regression(dfSPY,dfQQQ)
print(alphaBeta)
print(len(secondDF))

    Year     Alpha      Beta  Correlation
0   2010  0.000182  1.018034         True
1   2011  0.000050  0.973697         True
2   2012  0.000018  1.099745         True
3   2013  0.000139  0.971461         True
4   2014  0.000126  1.133949         True
5   2015  0.000327  1.092120         True
6   2016 -0.000199  1.124886         True
7   2017  0.000188  1.214837         True
8   2018  0.000229  1.268274         True
9   2019 -0.000012  1.225012         True
10  2020  0.000905  0.996076         True
11  2021 -0.000256  1.228431         True
12  2022 -0.000500  1.282122         True
13  2023  0.000615  1.251975         True
14  2024 -0.000228  1.349062         True
15  2025 -0.000972  1.308638         True
1258


In [39]:
def calculate_alpha_beta_val(alphaBeta):
    namda=0.95 #权重
    score=0
    for year in range(2010,2026):
        i=year-2010
        if(np.isnan(alphaBeta.loc[i,'Alpha'])):
            continue
            
        score+=np.power(namda,2025-year)*(alphaBeta.loc[i,'Alpha']+np.max(0,0.5-alphaBeta.loc[i,'Beta'])+0.1*np.max(0,alphaBeta.loc[i,'Correlation']-PEARSONBOUND))
        
    return score

In [40]:
def chooseGoodAlpha(df):
    goodETFs=[]
    SPYETF=pd.read_csv("../data/daily_data/SPY_daily.csv")
    for etf in df["Symbol"]:
        try :
            dfETF=pd.read_csv("../data/daily_data/"+etf+"_daily.csv")
        except Exception as e:
            print("Error: ",etf)
            print(e)
            continue
        if len(dfETF)<10:
            print("Empty: ",etf)
            continue
        # print(etf)
        # print(len(dfETF))
        # print(SPYETF.head())
        # print(dfETF.head())
        alphaBeta=linear_regression(SPYETF,dfETF)
        
        if calculate_alpha_beta_val(alphaBeta)>-0.0001:
            goodETFs.append(etf)
        
    ThirdDF=df[df["Symbol"].isin(goodETFs)]
    return ThirdDF

ThirdDF=chooseGoodAlpha(secondDF)
print(len(ThirdDF))
ThirdDF["Symbol"].to_csv("../data/ETF3.csv",index=False,header=False)

TypeError: 'numpy.float64' object cannot be interpreted as an integer